In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
# 헬퍼 라이브러리들
import numpy as np
import os, time
from utils import get_data

import IPython.display as ipd
from tensorflow.keras.layers import *
from tensorflow.keras import Model, Sequential
import tensorflow.keras.backend as K
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Import TensorFlow
import tensorflow as tf

generate_path = './generated_noise/'
feature = 'seq'
audio_path = '/root/datasets/ai_challenge/NOISEX/all/'
resample_sr = 16000
length = 4
train_data, train_label, label_list = get_data(feature=feature,resample_sr=resample_sr,length=length,audio_path=audio_path)

/root/anaconda3/envs/ten2.2/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/root/anaconda3/envs/ten2.2/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


1/45
2/45
3/45
4/45
5/45
6/45
7/45
8/45
9/45
10/45
11/45
12/45
13/45
14/45
15/45
16/45
17/45
18/45
19/45
20/45
21/45
22/45
23/45
24/45
25/45
26/45
27/45
28/45
29/45
30/45
31/45
32/45
33/45
34/45
35/45
36/45
37/45
38/45
39/45
40/45
41/45
42/45
43/45
44/45
45/45
data preprocessing complete, data feature is seq


In [2]:
BUFFER_SIZE = len(train_data)
class_num = len(label_list)
batch_size = 32
shape = train_data[0].shape
if len(shape) == 1:
    shape = (shape[0], 1)
EPOCHS = 100
noise_dim = 30
DATA_SHAPE = train_data[0].shape

train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label)).shuffle(BUFFER_SIZE).batch(batch_size)

def build_generator(output_shape=shape, class_num=class_num, stddev=0.2, z_dim=noise_dim):
    noise = Input(shape=(z_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(32, z_dim)(label))

    model_input = Concatenate()([noise, label_embedding])
    
    x = Dense(400, activation='relu')(model_input)
    x = tf.expand_dims(x, axis=1)
    x = LSTM(10, return_sequences=True)(x)
    x = Dense(output_shape[0], activation='tanh')(x)
    output = Reshape(output_shape)(x)

    return Model([noise, label], output)

def build_discriminator(input_shape=shape, class_num=class_num, stddev=0.2):
    noise_input = Input(shape=input_shape)
    reshaped_noise = Flatten()(noise_input)

    noise = Input(shape=input_shape)
    label = Input(shape=(1,), dtype='int32')

    label_embedding = Flatten()(Embedding(32, np.prod(input_shape))(label))
    flat_noise = Flatten()(noise)

    x = Concatenate()([flat_noise, label_embedding])
    x = tf.expand_dims(x, axis=-1)
    x = LSTM(10)(x)
    output = Dense(1, activation='sigmoid')(x)
    

    return Model([noise, label], output)

In [3]:
build_generator().summary()
build_discriminator().summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        960         input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 30)           0           embedding[0][0]                  
______________________________________________________________________________________________

In [ ]:
a = tf.constant([[1,2,3,4],[5,,6]])

In [4]:
# 체크포인트들을 저장하기 위해서 체크포인트 디렉토리를 생성합니다.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")


    # 또는 loss_fn = tf.keras.losses.sparse_categorical_crossentropy를 사용해도 됩니다.
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return tf.math.divide_no_nan(tf.math.add(real_loss,fake_loss),2)


gen_loss = tf.keras.metrics.Mean(name='gen_loss')
dis_loss = tf.keras.metrics.Mean(name='dis_loss')

dis_accuracy = tf.keras.metrics.BinaryAccuracy(name='dis_accuracy')

# 모델과 옵티마이저는 `strategy.scope`에서 만들어져야 합니다.
generator = build_generator()
discriminator = build_discriminator()
generator_optimizer = tf.keras.optimizers.Adam(0.001, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.001, 0.5)

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)
@tf.function
def train_step(noise, label):
    random_noise = tf.random.normal([noise.shape[0], noise_dim],dtype=tf.float32)
    noise = tf.cast(noise,dtype=tf.float32)
    label = tf.expand_dims(label, axis=1)
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_noise = generator([random_noise, label], training=True)

        real_output = discriminator([noise, label], training=True)
        fake_output = discriminator([generated_noise, label], training=True)

        g_loss = generator_loss(fake_output)
        d_loss = discriminator_loss(real_output, fake_output)
        gen_loss.update_state(g_loss)
        dis_loss.update_state(d_loss)
        
    gradients_of_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(d_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


    dis_accuracy.update_state(label, tf.math.divide_no_nan(tf.math.add(real_output, fake_output), 2))


In [ ]:

for epoch in range(EPOCHS):
    # 훈련 루프
    start = time.time()
    g_loss, step = 0., 0
    for noise_batch, label_batch in train_dataset:
        train_step(noise_batch,label_batch)
        step += 1


    if epoch % 2 == 0:
        checkpoint.save(checkpoint_prefix)
    print (f'{epoch}: d_loss: {dis_loss.result()}, d_accuracy: {dis_accuracy.result()*100}%, g_loss: {gen_loss.result()}',end=', ')
    print (f'time: {start - time.time()} seconds')

    gen_loss.reset_states()
    dis_accuracy.reset_states()
    dis_loss.reset_states()

In [ ]:
# eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
#       name='eval_accuracy')

# new_model = create_model()
# new_optimizer = tf.keras.optimizers.Adam()

# test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE)
# @tf.function
# def eval_step(images, labels):
#     predictions = new_model(images, training=False)
#     eval_accuracy(labels, predictions)
    
# checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# for images, labels in test_dataset:
#     eval_step(images, labels)

# print ('전략을 사용하지 않고, 저장된 모델을 복원한 후의 정확도: {}'.format(
#     eval_accuracy.result()*100))

gen = build_generator()
new_optimizer = Adam()
checkpoint = tf.train.Checkpoint(generator_optimizer=new_optimizer, generator=gen)
checkpoint.restore(tf.train.latest_checkpoint(checkpoin_dir))
def sample_noises(generator, epoch):
    noise = np.random.normal(0, 1, (class_num,noise_dim))
    sampled_labels = np.arange(0, class_num).reshape(-1,1)

    gen_sound = generator.predict([noise, sampled_labels])
    sampled_labels = np.arange(0, class_num).reshape(-1)
    for i, j in enumerate(sampled_labels):
        data = None
        if feature == 'stft':
            data = librosa.istft(gen_sound[i])
        elif feature == 'mfcc':
            data = librosa.feature.inverse.mfcc_to_audio(gen_sound[i].T, resample_sr)
        elif feature == 'seq':
            data = gen_sound[i]
        else:
            raise ValueError('wrong feature')

        librosa.output.write_wav(os.path.join(generate_path, f'{epoch}_{label_list[j]}.wav'), data, resample_sr, norm=True)